In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib3
import certifi
import re
from collections import defaultdict
from copy import copy, deepcopy

In [3]:
http = urllib3.PoolManager(
     cert_reqs='CERT_REQUIRED',
     ca_certs=certifi.where())

In [4]:
numery = []
with open('numery.txt') as f:
    numery = [int(n) for n in f.read().splitlines()]

In [5]:
def link_from_num(idd):
    return "http://mathgenealogy.org/id.php?id=" + str(idd)

In [265]:
for idd in numery:
    link = link_from_num(idd)
    page = http.request('GET', link, preload_content=False)
    soup = BeautifulSoup(page.read(), 'html.parser')
    mat_name = soup.find('h2')
    name = str(mat_name.string)
    break


Agnieszka  Badeńska 


In [6]:
def scrap_id(idd):
    link = link_from_num(idd)
    page = http.request('GET', link, preload_content=False)
    soup = BeautifulSoup(page.read(), 'html.parser')
    mat_name = soup.find('h2')
    try:
        name = str(mat_name.string).strip()
        st = mat_name.fetchNextSiblings()
    except Exception as exc:
        print(exc)
        print(idd)
        print(mat_name)
        print(soup)
    advisors = []
    for e in st:
        if str(e).find("Advisor") != -1:
            res = re.search(r"id=(\d*)", str(e))
            try:
                advisors.append(int(res.group(1)))
            except Exception as exc:
                if str(e).find("Unknown") != -1:
                    advisors.append(-1)
                else:
                    print("ERRRRROR")
                    print(exc)
                    print(e)
                    print(idd)
    d = dict()
    d['advisors'] = copy(advisors)
    d['name'] = name
    return d

In [7]:
db = defaultdict(dict)

for idd in numery:
    db[idd] = deepcopy(scrap_id(idd))

In [8]:
#db_hist = deepcopy(db)

In [9]:
#db = deepcopy(db_hist)

In [10]:
depth = 10
db_last = db
db_new = defaultdict(dict)
for i in range(depth):
    for k, d in db_last.items():
        for adv_id in d['advisors']:
            if not adv_id in db and not adv_id in db_new and adv_id != -1:
                db_new[adv_id] = deepcopy(scrap_id(adv_id))
    db = {**db, **db_new}
    db_last = deepcopy(db_new)

In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import pygraphviz as pgv
import pickle

In [34]:
db

{155901: {'advisors': [137002], 'name': 'Agnieszka  Badeńska'},
 168141: {'advisors': [133360], 'name': 'Czesław  Bagiński'},
 236278: {'advisors': [234593], 'name': 'Krystian Waldemar Baniak'},
 125996: {'advisors': [125944], 'name': 'Konstancja  Bobecka'},
 191656: {'advisors': [132016], 'name': 'Nadzieja  Borowik'},
 149083: {'advisors': [126075], 'name': 'T.  Brengos'},
 57480: {'advisors': [18596], 'name': 'Włodzimierz  Bryc'},
 185993: {'advisors': [125944], 'name': 'Joanna  Chachulska'},
 188601: {'advisors': [104953], 'name': 'Konrad  Ciecierski'},
 216475: {'advisors': [30182], 'name': 'Janusz  Czyż'},
 125997: {'advisors': [125944], 'name': 'Anna  Dembinska'},
 155731: {'advisors': [97165], 'name': 'Asen Lubomirov Dontchev'},
 124925: {'advisors': [124348], 'name': 'Ewa  Drabik'},
 108985: {'advisors': [75005], 'name': 'Andrzej  Dzielinski'},
 134439: {'advisors': [60324], 'name': 'Krystyna  Dzierzbicka-Mazaraki'},
 165964: {'advisors': [162216], 'name': 'Muktar Eltaher Elzob

In [29]:
graph = nx.DiGraph()

In [30]:
for mat_id, mat in db.items():
    for adv in mat['advisors']:
        if adv != -1:
            graph.add_edge(adv, mat_id)

In [31]:
graph_lab = nx.relabel_nodes(graph, {mat_id : mat['name'] for mat_id, mat in db.items()})

In [32]:
G = nx.nx_agraph.to_agraph(graph_lab)

In [33]:
G.layout()
G.draw(format='png', path='g.png')